In [6]:
import datetime

import pandas as pd
import tqdm

In [2]:
df_usage = pd.read_csv('../data/App_usage_trace.txt', sep=' ', names=['user', 'time', 'location', 'app', 'traffic'])
df_usage = df_usage[['user', 'time', 'app']]

# merging the consecutive usage records of the same app in one minute
df_usage['time'] = df_usage['time'].apply(lambda x: str(x)[:-2])
df_usage.drop_duplicates(inplace=True)

# delete apps used less than 10 times for all users
df_usage = df_usage[df_usage.groupby('app')['app'].transform('count').ge(10)]

In [3]:
prev_user = -1
prev_time = -1
app_seq = []
all_app_seq= []

seq_length = 4

recent_apps = []
all_recent_apps = []

# df_usage is already sorted based on user and time sequence

for i in tqdm.tqdm(range(len(df_usage))):
    user = df_usage.iloc[i]['user']
    app = df_usage.iloc[i]['app']
    time = df_usage.iloc[i]['time']
    time = datetime.datetime.strptime(time, '%Y%m%d%H%M')
    
    if prev_user != user:
        app_seq = [app]
        all_app_seq.append([])
        recent_apps = [app]
        all_recent_apps.append([])
        
    else:
        # same sequence if the time gap between them is equal to or less than 7 mins
        if (time - prev_time).total_seconds()//60 <= 7:
            if len(app_seq) == seq_length:
                all_app_seq.append(app_seq)
                app_seq = app_seq[1:] + [app]
                
            else:
                app_seq.append(app)
                all_app_seq.append([])
        else:
            app_seq = [app]
            all_app_seq.append([])
        
        # recently used apps
        all_recent_apps.append(recent_apps[:])

        recent_apps.append(app)
        if len(recent_apps) > 10:
            recent_apps.pop(0)
        
    
    prev_user = user
    prev_time = time

100%|██████████| 1123955/1123955 [06:30<00:00, 2877.10it/s]


In [4]:
df_usage['app_seq'] = all_app_seq
df_usage['recent_apps'] = all_recent_apps

# only filled sequences are treated as data
df_usage = df_usage[df_usage['app_seq'].map(len) != 0]
# delete users who have sequences less than 50
df_usage = df_usage[df_usage.groupby('user')['user'].transform('count').ge(50)]

df_usage.head()

,user,time,app,app_seq,recent_apps
8,0,201604200816,612,"[361, 361, 31, 360]","[361, 361, 31, 360]"
10,0,201604200817,31,"[361, 31, 360, 612]","[361, 361, 31, 360, 612]"
13,0,201604200817,360,"[31, 360, 612, 31]","[361, 361, 31, 360, 612, 31]"
14,0,201604200817,361,"[360, 612, 31, 360]","[361, 361, 31, 360, 612, 31, 360]"
16,0,201604200824,1,"[612, 31, 360, 361]","[361, 361, 31, 360, 612, 31, 360, 361]"


In [5]:
df_usage.to_csv('data/baseline.txt', sep='\t', index=False)